# Wine Classifier with KNN 

In [310]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_iris, fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform


import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from scipy.stats import randint





import warnings
warnings.filterwarnings('ignore')

In [311]:
df = pd.read_csv('../data/raw/winequality-red.csv', sep =';')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [312]:
df.shape

(1599, 12)

In [313]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [314]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed acidity,1599.0,8.319637,1.741096,4.60000,7.1000,7.90000,9.200000,15.90000
volatile acidity,1599.0,0.527821,0.179060,0.12000,0.3900,0.52000,0.640000,1.58000
citric acid,1599.0,0.270976,0.194801,0.00000,0.0900,0.26000,0.420000,1.00000
residual sugar,1599.0,2.538806,1.409928,0.90000,1.9000,2.20000,2.600000,15.50000
chlorides,1599.0,0.087467,0.047065,0.01200,0.0700,0.07900,0.090000,0.61100
free sulfur dioxide,1599.0,15.874922,10.460157,1.00000,7.0000,14.00000,21.000000,72.00000
total sulfur dioxide,1599.0,46.467792,32.895324,6.00000,22.0000,38.00000,62.000000,289.00000
density,1599.0,0.996747,0.001887,0.99007,0.9956,0.99675,0.997835,1.00369
pH,1599.0,3.311113,0.154386,2.74000,3.2100,3.31000,3.400000,4.01000
sulphates,1599.0,0.658149,0.169507,0.33000,0.5500,0.62000,0.730000,2.00000


## Elimino duplicados

In [315]:
df.duplicated()

0       False
1       False
2       False
3       False
4        True
        ...  
1594    False
1595    False
1596     True
1597    False
1598    False
Length: 1599, dtype: bool

## Limpieza de Datos

In [316]:
df.replace([np.inf, -np.inf], np.nan, inplace=True) 

df.dropna(inplace=True) #Elimino los valores nulos/NaN

In [317]:
df.duplicated() #Reviso si hay valores duplicados

0       False
1       False
2       False
3       False
4        True
        ...  
1594    False
1595    False
1596     True
1597    False
1598    False
Length: 1599, dtype: bool

## Elimino duplicados

In [318]:
df.drop_duplicates(inplace=True)

df = df.drop_duplicates()

df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
5,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1593,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [319]:
df = df.drop_duplicates()

# Split

In [320]:
def quality_to_label(q):
    if q <= 4:
        return 0  # low
    elif q <= 6:
        return 1  # medium
    else:
        return 2  # high

df['label'] = df['quality'].apply(quality_to_label)      

In [321]:
X = df.drop(["label", "quality"], axis=1)
y = df["label"]

In [322]:
df['label'].value_counts()

label
1    1112
2     184
0      63
Name: count, dtype: int64

In [323]:
X = df.drop("quality", axis=1)
y = df["quality"]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

X_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,label
1016,8.9,0.38,0.40,2.2,0.068,12.0,28.0,0.99486,3.27,0.75,12.6,2
1519,6.6,0.70,0.08,2.6,0.106,14.0,27.0,0.99665,3.44,0.58,10.2,1
452,6.8,0.56,0.03,1.7,0.084,18.0,35.0,0.99680,3.44,0.63,10.0,1
847,7.4,0.68,0.16,1.8,0.078,12.0,39.0,0.99770,3.50,0.70,9.9,1
58,7.8,0.59,0.18,2.3,0.076,17.0,54.0,0.99750,3.43,0.59,10.0,1


# Modelado & Escalado

In [324]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [325]:
model = KNeighborsClassifier()

model.fit(X_train, y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


# Predicción

In [326]:
y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)

y_pred_test

array([6, 6, 7, 5, 6, 7, 6, 6, 5, 5, 6, 5, 6, 5, 6, 5, 6, 5, 6, 5, 5, 5,
       5, 5, 5, 5, 6, 5, 6, 6, 5, 6, 6, 5, 6, 5, 5, 5, 5, 5, 5, 5, 6, 5,
       5, 6, 5, 5, 5, 5, 5, 5, 6, 6, 6, 5, 6, 5, 5, 7, 5, 6, 5, 6, 5, 6,
       5, 6, 6, 7, 5, 5, 6, 5, 5, 5, 5, 6, 5, 6, 6, 5, 6, 6, 6, 6, 5, 6,
       6, 6, 6, 5, 5, 6, 6, 5, 4, 7, 5, 6, 5, 6, 5, 5, 5, 6, 5, 6, 6, 6,
       6, 6, 5, 6, 5, 5, 6, 6, 5, 5, 5, 6, 5, 5, 6, 6, 6, 6, 6, 5, 6, 5,
       5, 5, 5, 5, 7, 6, 6, 5, 5, 5, 7, 7, 5, 5, 6, 5, 6, 5, 5, 6, 5, 4,
       5, 6, 7, 6, 6, 5, 5, 5, 5, 6, 6, 6, 5, 5, 5, 6, 5, 6, 6, 5, 6, 6,
       6, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 5, 6, 6, 6, 5, 7, 5,
       6, 6, 5, 6, 5, 6, 5, 5, 5, 6, 6, 5, 7, 5, 6, 5, 6, 5, 5, 6, 5, 6,
       6, 6, 5, 6, 6, 6, 7, 5, 5, 5, 5, 6, 7, 5, 6, 6, 6, 6, 5, 6, 6, 5,
       5, 5, 7, 5, 6, 5, 5, 7, 5, 5, 5, 6, 6, 5, 5, 5, 5, 5, 5, 6, 6, 5,
       5, 5, 6, 7, 6, 6, 6, 6])

In [327]:
accuracy = {'Accuracy Test': accuracy_score(y_test, y_pred_test),
            'Accuracy Train': accuracy_score(y_train, y_pred_train)}
accuracy

{'Accuracy Test': 0.47794117647058826, 'Accuracy Train': 0.6614535418583257}

### Hiperparametrización

In [328]:
hiper_params = {'n_neighbors': [11, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35],
                'weights': ['uniform', 'distance'],
                'p': [1, 2]}

In [329]:
random_search = RandomizedSearchCV(estimator=model,
                                   param_distributions=hiper_params,
                                   n_iter=50,
                                   cv=5,
                                   scoring='accuracy',
                                   random_state=42,
                                   n_jobs=-1)

random_search.fit(X_train, y_train)
random_search.best_params_

{'weights': 'distance', 'p': 1, 'n_neighbors': 23}

In [330]:
random_search.best_score_

np.float64(0.582332050902634)

In [331]:
best_knn = random_search.best_estimator_
y_pred = best_knn.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
test_accuracy

0.5294117647058824